In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Msc_Test/ACDC_Sample_Data

/content/drive/MyDrive/Msc_Test/ACDC_Sample_Data


In [3]:
#Viewing distorition

In [7]:
import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import imutils
import random

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

images = os.listdir(input_path)
print(images)
for image in images:
	if image.endswith("view.png"):
		print("working on files",image)
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		image2 = np.zeros((256,256))
		h,w = image2.shape
		num1 = random.randint(0, 3)
		area = random.randint(0, 20)

		if num1 == 0: #crop left
			image2[:,0:w-area] = image1[:,area:]
		elif num1 == 1: #crop bottom
			image2[area:,:] = image1[0:h-area,:]
		elif num1 == 2: #crop right
			image2[:,area:] = image1[:,0:w-area]
		elif num1 == 3: #crop top
			image2[0:h-area,:] = image1[area:,:]

		cv2.imwrite(os.path.join(output_path,image),image2)

['patient001_frame01_2_lessghosting.png', 'patient001_frame01_2_motion.png', 'patient001_frame01_2_view.png', 'patient001_frame01_2_motionGhosting.png', '.DS_Store', 'patient001_frame01_2_moreghosting.png', 'patient001_frame01_2_missing.png']
working on files


In [ ]:
#Ghosting and motion

In [8]:
import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import imutils

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

def calculate_2dft(input):
	ft = np.fft.fft2(input)
	return np.fft.fftshift(ft)

def calculate_2dift(input):
	ift = np.fft.ifft2(input)
	return ift

images = os.listdir(input_path)
for image in images:
	if image.endswith("motionGhosting.png"):
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		h,w = image1.shape
		dimw = int(w/2)
		dimh = int(h/2)
		image2 = np.zeros(image1.shape,dtype="uint8")
		image2[0:h-10,:] = image1[10:,:]
		image3 = np.zeros(image1.shape,dtype="uint8")
		image3[10:,:] = image1[0:h-10,:]
		img3 = cv2.addWeighted(src1=image1[:,0:dimw], alpha=0.85, src2=image2[:,0:dimw], beta=0.15, gamma=0.0)
		img3 = cv2.addWeighted(src1=img3, alpha=0.85, src2=image3[:,0:dimw], beta=0.15, gamma=0.0)
		image1[:,0:dimw] = img3

		image2 = image1.copy()
		h,w = image1.shape
		midh = int(h/2)
		midw = int(w/2)
		image1 = imutils.rotate(image1, -1)
		image2 = imutils.rotate(image2, +1)
		ft1 = calculate_2dft(image1)
		ft2 = calculate_2dft(image2)
		final = np.zeros(ft1.shape,dtype="complex128")
		final[0:midh,0:midw] = ft1[0:midh,0:midw]
		final[midh:,midw:] = ft1[midh:,midw:]
		final[0:midh,midw:] = ft2[0:midh,midw:]
		final[midh:,0:midw] = ft2[midh:,0:midw]
		ift = calculate_2dift(final)
		recon = (abs(ift))
		cv2.imwrite(os.path.join(output_path,image),recon)



In [ ]:
# motion

In [9]:
import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import imutils

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

def calculate_2dft(input):
	ft = np.fft.fft2(input)
	return np.fft.fftshift(ft)

def calculate_2dift(input):
	ift = np.fft.ifft2(input)
	return ift

images = os.listdir(input_path)
for image in images:
	if image.endswith("motion.png"):
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		image2 = image1.copy()
		h,w = image1.shape
		midh = int(h/2)
		midw = int(w/2)
		image1 = imutils.rotate(image1, -3)
		image2 = imutils.rotate(image2, +3)
		ft1 = calculate_2dft(image1)
		ft2 = calculate_2dft(image2)
		final = np.zeros(ft1.shape,dtype="complex128")
		final[0:midh,0:midw] = ft1[0:midh,0:midw]
		final[midh:,midw:] = ft1[midh:,midw:]
		final[0:midh,midw:] = ft2[0:midh,midw:]
		final[midh:,0:midw] = ft2[midh:,0:midw]
		ift = calculate_2dift(final)
		recon = (abs(ift))
		cv2.imwrite(os.path.join(output_path,image),recon)



In [ ]:
# missing patch

In [10]:

import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import imutils
import random

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

images = os.listdir(input_path)
for image in images:
	if image.endswith("missing.png"):
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		image1[110:120,110:120] = 0
		image1[80:90,100:110] = 0
		image1[40:50,200:210] = 0
		cv2.imwrite(os.path.join(output_path,image),image1)

In [ ]:
#less ghosting

In [11]:

import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

images = os.listdir(input_path)
for image in images:
	if image.endswith("lessghosting.png"):
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		h,w = image1.shape
		image2 = np.zeros(image1.shape,dtype="uint8")
		image2[0:h-20,:] = image1[20:,:]
		image3 = np.zeros(image1.shape,dtype="uint8")
		image3[20:,:] = image1[0:h-20,:]
		img3 = cv2.addWeighted(src1=image1, alpha=0.80, src2=image2, beta=0.20, gamma=0.0)
		img3 = cv2.addWeighted(src1=img3, alpha=0.80, src2=image3, beta=0.20, gamma=0.0)
		cv2.imwrite(os.path.join(output_path,image),img3)

In [ ]:
# more ghosting

In [12]:

import cv2
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

input_path = "/content/drive/MyDrive/mri/Sample_input"
output_path = "/content/drive/MyDrive/mri/final_test"

images = os.listdir(input_path)
for image in images:
	if image.endswith("moreghosting.png"):
		image1 = cv2.imread(os.path.join(input_path,image))
		image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
		image2 = np.zeros(image1.shape,dtype="uint8")
		image3 = np.zeros(image1.shape,dtype="uint8")
		h,w = image1.shape
		dimh = int(h/2)
		image2[0:dimh,:] = image1[dimh:,:]
		image3[dimh:,:] = image1[0:dimh,:]
		img3 = cv2.addWeighted(src1=image1, alpha=0.80, src2=image2, beta=0.20, gamma=0.0)
		img3 = cv2.addWeighted(src1=img3, alpha=0.80, src2=image3, beta=0.20, gamma=0.0)
		cv2.imwrite(os.path.join(output_path,image),img3)
